# comparison with plume tracker outputs

because

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import xarray as xr

from utils import create_gif
from parcels_utils import xr_dataset_to_fieldset, get_file_info
from plot_utils import get_carree_axis, plot_particles

In [ ]:
TIJUANA_RIVER_DOMAIN = dict(
    S=32.528,
    N=32.71,
    W=-117.29,
    E=-117.11,
)
CLOSE_TIJUANA_DOMAIN = dict(
    S=32.53,
    N=32.5825,
    W=-117.162,
    E=-117.105,
)
THING_DOMAIN = {
    "S": 32.41,
    "N": 32.7,
    "W": -117.25,
    "E": -117
}

In [ ]:
def parse_mat_time(time):
    date = np.datetime64("0") + np.timedelta64(int(time) - 1, "D")
    hour = int(round((time - int(time)) * 24))
    return date + np.timedelta64(hour, "h")


def dataset_from_mat(grid_mat_path, current_mat_path):
    grid_mat = scipy.io.loadmat(grid_mat_path)
    current_mat = scipy.io.loadmat(current_mat_path)
    coords = grid_mat["totalGrid"]
    u = current_mat["U"]
    v = current_mat["V"]
    time = np.array([parse_mat_time(current_mat["t"][0, 0])])
    lats = np.sort(np.unique(coords.T[1]))
    lons = np.sort(np.unique(coords.T[0]))
    u_grid = np.zeros((1, len(lats), len(lons)))
    v_grid = np.zeros((1, len(lats), len(lons)))
    for i in range(len(lons)):
        for j in range(len(lats)):
            # data from .mat is measured in cm/s
            u_grid[0, j, i] = u[0][i * len(lats) + j] / 100
            v_grid[0, j, i] = v[0][i * len(lats) + j] / 100
    ds = xr.Dataset(
        {
            "u": (["time", "lat", "lon"], u_grid),
            "v": (["time", "lat", "lon"], v_grid),
        },
        coords={
            "time": time,
            "lat": lats,
            "lon": lons
        }
    )
    return ds

In [ ]:
ds = dataset_from_mat("mat/codartotalGrid.mat", "mat/Tot_SDLJ_20200810_0000.mat")
ds

In [ ]:
fs = xr_dataset_to_fieldset(ds)

In [ ]:
thredds = get_file_info("current_netcdfs/west_coast_1km_hourly/tj_plume_interped.nc", 1, name="target")
thredds["xrds"]

In [ ]:
fs.U.show()
thredds["fs"].U.show()

In [ ]:
data = scipy.io.loadmat("mat/glist90zj_pts_position.mat")
plot_particles(
    data["yf"], data["xf"], None, THING_DOMAIN,
    show=False, savefile=f"mat/snap/glishsnap.png", part_size=1
)

In [ ]:
file_prefix = "pts_20200810_"
start = 30
incr = 100

In [ ]:
for i in range(start, 2400, incr):
    name = f"mat/{file_prefix}{str(i).zfill(4)}.mat"
    try:
        data = scipy.io.loadmat(name)
        plot_particles(
            data["yf"], data["xf"], None, THING_DOMAIN,
            show=False, savefile=f"mat/snap/s{str(i).zfill(4)}.png", part_size=1,
            titlestr=f"particles at {i}"
        )
    except FileNotFoundError:
        print(f"file {name} not found")

In [ ]:
create_gif(25, "mat/snap/s*.png", "mat/g.gif")